In [1]:
import numpy as np
import os
from tqdm import trange
from argparse import Namespace
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow import keras as k
from _keras import data as D
from _keras import architectures as A
import utils as u
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
import albumentations as albu

# Plotting library
from matplotlib import pyplot as plt
from matplotlib import rcParams
from mpl_toolkits.axes_grid1 import make_axes_locatable

rcParams.update({
    'image.cmap'     : 'gray',
    'axes.grid'      : False,
    'savefig.dpi'    : 300,  # to adjust notebook inline plot size
    'axes.labelsize' : 14,  # fontsize for x and y labels (was 10)
    'axes.titlesize' : 14,
    'font.size'      : 14,
    'legend.fontsize': 14,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'text.usetex'    : False
})

In [19]:
args = Namespace(
    dataset='train8_xz.csv',
    gpu=5,
    nf=32,
    batch=16,
    backbone='xception',
    use_inputs=True,
    use_edges=False,
    separate_classes=False,
    hflip=True,
    downscale=True,
    shuffle=True,
    
)

In [3]:
u.set_gpu(args.gpu)

GPU selected: 5 - TITAN V


In [4]:
data_root = "/nas/home/fpicetti/datasets/seismic_facies/"
dataset = os.path.join(data_root, args.dataset)
num_classes=6

In [20]:
aug = []
if args.hflip:
    aug.append(albu.HorizontalFlip())
if args.downscale:
    aug.append(albu.Downscale(scale_min=0.5, scale_max=0.95, p=0.1))
        
datagen_train = D.CSVLoader(
    csv_file=dataset,
    phase="train",
    batch_size=args.batch,
    separate_classes=args.separate_classes,
    use_edges=args.use_edges,
    use_inputs=args.use_inputs,
    shuffle=args.shuffle,
    augmentation=albu.Compose(aug),
    preprocessing=None,
)

In [21]:
patch_shape = datagen_train[0][0].shape[1:]

In [22]:
# Build model
K.clear_session()

model = A.MultiDecoder(
    patch_shape,
    num_classes=num_classes,
    separate_classes=args.separate_classes,
    use_edges=args.use_edges,
    use_inputs=args.use_inputs,
    nf=args.nf, act="relu",
    backbone=args.backbone)

In [23]:
model.summary()

Model: "MultiDecoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder (Model)              (None, 62, 8, 256)        183456    
_________________________________________________________________
Segmenter (Model)            (None, 992, 128, 6)       1875814   
_________________________________________________________________
Reconstructor (Model)        (None, 992, 128, 1)       1874369   
Total params: 3,933,639
Trainable params: 3,927,943
Non-trainable params: 5,696
_________________________________________________________________


In [29]:
model.segmenter.trainable = False

In [24]:
model.compile(loss='categorical_hinge', optimizer=k.optimizers.Adam(lr=0.01))

In [45]:
image, mask = datagen_train[2]

In [46]:
losses = model.train_on_batch(image, mask)

In [47]:
losses

[1.248868703842163, 1.1945821046829224, 0.05428662523627281]

In [28]:
model.metrics_names

['loss', 'output_1_loss', 'output_2_loss']